In [ ]:
import sys
sys.path.append("../")
import argparse
import pandas as pd
from torch.utils.data import DataLoader
from tikuna.models import LSTM
from tikuna.common.preprocess import FeatureExtractor
from tikuna.common.dataloader import load_sessions, log_dataset
from tikuna.common.utils import seed_everything, dump_final_results, dump_params, load_params

In [ ]:
NORMAL_LOG_DATA="/home/tikuna/app/data/mainnet/normal/log/normal.log"
ECLIPSE_LOG_DATA="/home/tikuna/app/data/mainnet/eclipse-single/log/eclipse.log"

In [ ]:
abnormal_data = pd.read_csv(ECLIPSE_LOG_DATA,
                         sep = '\s+',
                         names=["Timestamp", "Removed IP", "Removed Port",
                                "Added IP", "Added Port", "Bucket", "label"])
display(abnormal_data)
abnormal_data['Timestamp'] = pd.to_datetime(abnormal_data['Timestamp'],
                                          format='[%Y-%m-%d|%H:%M:%S.%f]')
evaluation_data = {}

evaluation_data["features"] = abnormal_data.iloc[:, 1:6]
evaluation_data["label"] = abnormal_data.iloc[:, [6]].replace("abnormal", 1)
evaluation_data["type"] = "testing"

In [ ]:
params, meta_data = load_params("current")
params["cache"] = True

seed_everything(params["random_seed"])

ext = FeatureExtractor(**params)

session_test = ext.fit_transform(evaluation_data, datatype="test")
dataset_test = log_dataset(session_test, feature_type=params["feature_type"])
dataloader_test = DataLoader(
    dataset_test, batch_size=100, shuffle=False, pin_memory=True
)

model_save_path = dump_params(params, meta_data)
model = LSTM(meta_data=meta_data, model_save_path=model_save_path, **params)
model.label_type = "anomaly"
final_test_results = model.evaluate(
    dataloader_test,
    dtype="evaluate"
)